### Fonte: https://appliedmachinelearning.blog/2018/01/18/conventional-approach-to-text-classification-clustering-using-k-nearest-neighbor-k-means-python-implementation/

### Importando módulos

In [1]:
import glob, os, scipy, nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem.wordnet import WordNetLemmatizer
import re
from sklearn.preprocessing import OneHotEncoder


### Criando as funções

In [30]:
def formatText(text):
    #Esta função tem o objetivo de formatar o texto para o classificador K Nearest Neighborhud - KNN
    # 1 - Converter string em lista
    # 2.1 - Remover as stopwords
    # 2.2 - Remover as pontuações
    # 3 - Remover os sufixos das palavras
        
    #Retirando espacos e colando as palavras minuscula
    wpt = WordPunctTokenizer()
    text = np.array(text)
    
    
    #text = (text.str.lower()).apply(wpt.tokenize)#wpt.tokenize(text.lower()) 
    text = map(wpt.tokenize,text)    
    #Retirando as stopwords e as pontuacoes
    #Coletando as stopword em Português
    stopw = stopwords.words('english') + list(punctuation)
    
    #Retirando as stopwords
    palavras_sem_stopwords = []
    for word_final in text:
        palavras_sem_stopwords.append([word for word in word_final if word not in stopw])

    #Retirando os sufixos das palavras
    stm = PorterStemmer()
    texto_final_matrix = list()
    for texto in palavras_sem_stopwords:    
        texto_final = [stm.stem(txt.lower()) for txt in texto]
        texto_final_matrix.append(texto_final)
    
    text = texto_final_matrix

    listText = [] 
    for row in text:
        a = ''
        for txt in row:
            a += ' '+txt
        listText.append(a)
    
    text = listText
    return text

#Criar um dataframe a partir de dados em formato .txt
def preProcessingCorpus(folder):

    #Criando o arquivo
    files = glob.glob(os.path.join(folder,'*.txt'))
    
    #Criando o nova dataframe
    df = pd.DataFrame()
    count = 1   
    #Percorrendo todos os arquivo
    for file in files:      
    
        #Coletando informacoes do contidas no arquivo e nome do arquivo com docid
        data = open(file,'r',encoding='utf-8')
        docid = str(os.path.basename(file)[0:-4])
            
        #Inserindo valores no dataframe      
        datas =pd.DataFrame({'id':[count],'docid':[docid],'text':[data.read()],'class1':[0],'class2':[0]})
        
        df = df.append(datas)
        datas = None
    
        #Fechando o arquivo
        data.close()
        count += 1
    
    return df
           

In [38]:
#Leitura do arquivo
foldersrc = r'<local dos arquivos de treino>'
os.chdir(foldersrc)

#Criando o nova dataframe
df = pd.DataFrame()
count = 1

for x in os.listdir('.'):
    if os.path.isdir(x):
        
        #Criando o folder
        subfolder = os.path.join(foldersrc,os.path.join(x,'*.txt'))
               
        #Criando o arquivo
        files = glob.glob(subfolder)
       
        #Percorrendo todos os arquivo
        for file in files:      
    
            #Coletando informacoes do contidas no arquivo e nome do arquivo com docid
            data = open(file,'r',encoding='utf-8')
            docid = str(os.path.basename(file)[0:-4])
            
            #Inserindo valores no dataframe      
            datas =pd.DataFrame({'id':[count],'docid':[docid],'text':[data.read()],'class':[int(x)-1]})
        
            df = df.append(datas)
            datas = None
    
            #Fechando o arquivo
            data.close()
            count += 1




### Separando amostras e rótulos

In [27]:
#Formatando o texto
df['text'] = formatText(df['text'])
vectorizer = TfidfVectorizer()

#Separando as variaveis independentes das dependentes
independentes = vectorizer.fit_transform(df['text'])
dependentes = np.array(df['class'])


### Classifcando textos

In [36]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(independentes,dependentes)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

### Preditar o modelo

In [37]:
src = r'<pasta do arquivo para preditar>'
df_pred_data = preProcessingCorpus(src)
df_pred_data['text'] = formatText(df_pred_data['text'])
pred_data = vectorizer.transform(df_pred_data['text'])
labels_preditos = knn.predict_proba(pred_data)
rows,cols = labels_preditos.shape
df_pred_data['class1'] = labels_preditos[np.array(range(rows)),0] #Class 1
df_pred_data['class2'] = labels_preditos[np.array(range(rows)),1] #Class 2

writer = pd.ExcelWriter('<saida do arquivo>')
df_pred_data.to_excel(writer,'<nome da planilha>')
writer.save()
  
